In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
#!python -m spacy download es_core_news_lg

In [3]:
df = pd.read_csv('./final_bd_tv_ner_all.csv')
df.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v,Noblex,9,15
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v,Tcl,9,12
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos,Samsung,3,10
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota,Samsung,7,14
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon,Sanyo,6,11


In [5]:
marcas = pd.read_csv('./tv_marcas_meli.csv')
marcas.head()

,marca
0,Admiral
1,AOC
2,Audinac
3,Audiologic
4,BGH


In [6]:
import spacy
#from spacy import displacy

nlp = spacy.load('es_core_news_lg', disable = ['tagger','ner'])

In [8]:
#Vectores Marcas
list_vec = marcas['marca'].to_list()
marcas_vec = [nlp(x) for x in list_vec]

In [9]:
# Vectores NER
list_vec = df['marca'].to_list()
ner_vec = [nlp(x) for x in list_vec]

In [10]:
clean_ents = []

for ner in ner_vec:
    paso = 0
    for marca in marcas_vec:
        #print("Lista ruler: ", vec)
        similarity = marca.similarity(ner)
        #print(similarity)
        #print("Lista ner: ", text1)
        #print("........")                                        
        if similarity > 0.90:
            #print(marca.text)
            clean_ents.append(marca.text)
            paso = 1
            break
    if paso == 0:
        clean_ents.append(ner.text)    


/tmp/ipykernel_18442/1658257045.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = marca.similarity(ner)


In [11]:
pd.Series(clean_ents).value_counts()

Samsung     3348
Philips     2402
LG          1814
##REV       1699
Noblex      1134
            ... 
Pilihps        1
Hyndai         1
E LG           1
Lg55           1
Servicio       1
Length: 244, dtype: int64

In [12]:
pd.Series(list_vec).value_counts()

Samsung     3348
Philips     2402
LG          1814
##REV       1699
Noblex      1133
            ... 
Pilihps        1
Hyndai         1
E LG           1
Lg55           1
Servicio       1
Length: 251, dtype: int64

# Fastext + Spacy
# Por experiencia previa decido no hacerla...

In [75]:
#!python -m spacy init-model es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/models/cel_model_vec' --vectors-loc '/home/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec'

✔ Successfully created model
3067it [00:00, 32252.47it/s]/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec
✔ Loaded vectors from
/home/fede/fastText-0.9.2/result/nubi_cel_vectors_300.vec
✔ Sucessfully compiled vocab
3293 entries, 3067 vectors


# Fastext

* %cd '/home/fede/fastText-0.9.2'
* !pip install fastText
* !./fasttext
* !./fasttext skipgram -input ./training_data/nubi_tv.jsonl -output models/fasttext/nubi_tv_vectors_300 -dim 300

In [13]:
import fasttext

In [14]:
model = fasttext.load_model('./models/fasttext/nubi_tv_vectors_300.bin')

In [15]:
fast_m = model.get_nearest_neighbors('Samsum', k=10)
fast_m

[(0.9969042539596558, 'Samsumg'),
 (0.9968076944351196, 'Samsung,'),
 (0.9966309666633606, 'Samnsung'),
 (0.9951133728027344, 'Samsung.'),
 (0.9925115704536438, 'Rack'),
 (0.9923394322395325, 'Una'),
 (0.9918062686920166, 'Garantía'),
 (0.9916898608207703, 'Curve'),
 (0.9915932416915894, 'Back'),
 (0.9915770292282104, 'Terrace')]

## Decido entrenar un modelo con todos los datos.

In [21]:
model = fasttext.load_model('./models/fasttext/nubi_cel_tv_vectors_300.bin')

In [24]:
fast_m = model.get_nearest_neighbors('Audio logic', k=10)
fast_m

[(0.9888470768928528, 'Audiologic'),
 (0.9661963582038879, 'Picture'),
 (0.9626744985580444, 'Audio'),
 (0.9622519612312317, 'Audinac'),
 (0.9616380333900452, "'"),
 (0.9607887268066406, 'Stereo'),
 (0.9604459404945374, 'Firstline'),
 (0.9599198698997498, 'Audiovox'),
 (0.957794725894928, 'Multinorma'),
 (0.9495194554328918, 'Ranser')]

In [25]:
#Vectores Marcas
marcas = marcas['marca'].to_list()
ents = df['marca'].to_list()

In [26]:
marc_ents = []
count = 0
for ent in ents:
    paso = 0   
    
    marc = [x for x in marcas if ent.lower() in x.lower()]
    
    if marc:
        marc_ents.append(marc[0])
        paso = 1
    else:
        ent_neigs = model.get_nearest_neighbors(ent, k=10) 
        for ent_neig in ent_neigs:
            if ent_neig[0] > 0.95:
                marc_ = [x for x in marcas if ent_neig[1].lower() in x.lower()]            
                if marc_:
                    marc_ents.append(marc_[0])
                    #print(ent_neig[1])
                    paso = 1
                    break            
    if paso == 0:
        marc_ents.append(ent)
        count+=1
print(count)

1991


In [81]:
# new_list = [x for x in marcas if "Zte".lower() in x.lower()]
# new_list

In [27]:
pd.Series(marc_ents).value_counts()

Samsung      3368
Philips      2680
LG           1815
##REV        1699
Noblex       1137
             ... 
Golpeó          1
Orient          1
Tiendaaps       1
Kenmark         1
Servicio        1
Length: 170, dtype: int64

In [28]:
pd.Series(ents).value_counts()

Samsung     3348
Philips     2402
LG          1814
##REV       1699
Noblex      1133
            ... 
Pilihps        1
Hyndai         1
E LG           1
Lg55           1
Servicio       1
Length: 251, dtype: int64

In [29]:
df['marcas_corregidas'] = marc_ents
df.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end,marcas_corregidas
0,0,b10532ee783f9545d7bddd8f9da20d2a8d1642a486caf5...,63c875becd6c78649539497bdd134c2c762f610b3616ae...,Smart Tv Noblex Di43x5100x Led Full Hd 43 220v,Noblex,9,15,Noblex
1,1,aa4dd7ba40c80cf15a859c19f2500e328797e2251322d4...,490e4b75cfa66087c95b20fb6546e4dfdc952f7537a8e3...,Smart Tv Tcl L42s6500 Led Full Hd 42 220v,Tcl,9,12,TCL
2,2,e48526711c7cd386af46750540bf107acac9b5988b515a...,eab7301d49cd3bbe3bcd3c196da0d61713de45b44523c1...,Tv Samsung Smart Tv Hd 32 Para Repuestos,Samsung,3,10,Samsung
3,3,9e527f7876123f50719291aed05c8351bab1ae9abacadb...,daf900de69964f2cf241ed5395ac74edd1f2deb6577356...,Tv Led Samsung 46 Smart Pantalla Rota,Samsung,7,14,Samsung
4,4,cf3021aa458178431fa2f22daa52ee17f005d6728d9b35...,b244c68806fab7d7d523a3927a4eb6381610c40313e8bb...,Tv 29 Sanyo Vizon,Sanyo,6,11,Sanyo


In [96]:
df.to_csv("./tv_bd.csv", index=False)